In [1]:
import os
import pandas
from pandas import read_csv
import json
import pickle
from datetime import datetime
import sys
import PyPDF2 as pypdf
import requests

## Parse PDFs for initial dump

In [2]:
#### Create curatedBy Object
def generate_curator():
    todate = datetime.now()
    curatedByObject = {"@type": "Organization", "identifier": "covid19LST", "url": "https://www.covid19lst.org/", 
                              "name": "COVID-19 Literature Surveillance Team", "affiliation": [], 
                              "curationDate": todate.strftime("%Y-%m-%d")}
    return(curatedByObject)

In [3]:
def generate_author():
    authorObject = generate_curator()
    authorObject.pop('curationDate')
    memberlist = read_csv('data/LST members.txt',delimiter='\t',header=0,encoding='UTF-8')
    memberlist.rename(columns={'affiliation':'affiliation list'}, inplace=True)
    memberlist['affiliation']='blank'
    for i in range(len(memberlist)):
        affiliationlist = memberlist.iloc[i]['affiliation list'].split(';')
        tmplist = []
        for eachaffiliation in affiliationlist:
            tmplist.append({"name":eachaffiliation})
        memberlist.at[i,'affiliation'] = tmplist
    memberlist.drop(columns='affiliation list',inplace=True)
    memberdictlist = memberlist.to_dict('records')
    authorObject['members']=memberdictlist 
    return(authorObject)

In [43]:
def generate_abstract(publist):
    cleanlist = [str(item) for item in publist]
    separator = ', '
    abstract = "Analytical reviews on the level of evidence presented in publications. This report specifically covers the following publications: "+ separator.join(cleanlist)
    return(abstract)

In [5]:
### Batch convert DOIs
def convert_dois(doilist):
    doistring = '"' + '","'.join(doilist) + '"'
    r = requests.post("https://api.outbreak.info/resources/query/", params = {'q': doistring, 'scopes': 'doi', 'fields': '_id,name,url,doi'})
    if r.status_code == 200:
        rawresult = pandas.read_json(r.text)
        cleanresult = rawresult.drop_duplicates(subset='_id',keep="first")
        if len(doilist)>len(cleanresult):
            missing = [x for x in doilist if x not in cleanresult['doi'].unique().tolist()]
        else:
            missing = None
        cleanresult.drop(columns=['doi','_score','query'],inplace=True)
    return(cleanresult, missing)

### Convert a single doi
##"https://api.outbreak.info/resources/query?q=doi:"+doi

### Batch fetch pmid meta
def get_pmid_meta(pmidlist):
    pmidstring = '"' + '","'.join(pmidlist) + '"'
    r = requests.post("https://api.outbreak.info/resources/query/", params = {'q': pmidstring, 'scopes': '_id', 'fields': '_id,name,url'})
    if r.status_code == 200:
        rawresult = pandas.read_json(r.text)
        cleanresult = rawresult[['_id','name','url']].loc[rawresult['_score']==1].copy()
        cleanresult.drop_duplicates(subset='_id',keep="first", inplace=True)
        if len(pmidlist)>len(cleanresult):
            missing = [x for x in pmidlist if x not in cleanresult['_id'].unique().tolist()]
        else:
            missing = None
    return(cleanresult, missing)    


In [30]:
def parse_pdf(eachfile):
    try:
        pdffile = open('data/reports/'+eachfile,'rb')
        pdf = pypdf.PdfFileReader(pdffile)
        pages = pdf.getNumPages()
        key = '/Annots'
        uri = '/URI'
        ank = '/A'
        allurls = []
        for page in range(pages):
            pageSliced = pdf.getPage(page)
            pageObject = pageSliced.getObject()
            if key in pageObject:
                ann = pageObject[key]
                for a in ann:
                    u = a.getObject()
                    if ank in u:
                        if uri in u[ank]:
                            allurls.append(u[ank][uri])
        pmidlist = []
        doilist = []
        for eachurl in allurls:
            if 'pubmed' in eachurl and '?' not in eachurl:
                pmid = eachurl.replace("https://www.ncbi.nlm.nih.gov/pubmed/","").replace("https://pubmed.ncbi.nlm.nih.gov/","").rstrip("/")
                if "#affiliation" in pmid:
                    trupmid = pmid.split("/")[0]
                    tmpid = 'pmid'+trupmid
                else:
                    tmpid = 'pmid'+pmid
                pmidlist.append(tmpid)
            elif 'doi' in eachurl:
                tenplace = eachurl.find('10.')
                doi = eachurl[tenplace:]
                doilist.append(doi)
            pmidlist = list(set(pmidlist))
            doilist = list(set(doilist))  
    except:
        pmidlist = []
        doilist = []
    return(pmidlist,doilist)

def merge_meta(pmidlist,doilist):
    if len(doilist)>0:
        doianns,missing_dois = convert_dois(doilist)
        doicheck = True
    else:
        doicheck = False
        missing_dois = None
    if len(pmidlist)>0:
        pmidanns,missing_pmids = get_pmid_meta(pmidlist)
        pmidcheck = True
    else:
        pmidcheck = False
        missing_pmids = None
    if doicheck==True and pmidcheck==True:
        basedOndf = pandas.concat((pmidanns,doianns),ignore_index=True)
    elif doicheck==True and pmidcheck==False:
        basedOndf = doianns
    elif doicheck==False and pmidcheck==True:
        basedOndf = pmidanns     
    if missing_pmids!=None and missing_dois!=None:
        missing = list(set(missing_pmids).union(set(missing_dois)))
    elif missing_pmids==None and missing_dois!=None:
        missing = missing_dois
    elif missing_pmids!=None and missing_dois==None:
        missing = missing_pmids
    else:
        missing = None
    return(basedOndf,missing)

In [7]:
def save_missing(missing):
    try:
        missing_list = pickle.load(open('results/pubs_not_yet_in_outbreak.txt','rb'))
        if missing != None:
            total_missing = list(set([*missing_list, *missing]))
            with open('results/pubs_not_yet_in_outbreak.txt','wb') as dmpfile:
                pickle.dump(total_missing,dmpfile)
    except:
        if missing != None:
            with open('results/pubs_not_yet_in_outbreak.txt','wb') as dmpfile:
                pickle.dump(missing,dmpfile)

In [8]:
## Note that strftime("%d") will give the day with a leading zero
## In windows, strftime("%#d") will give it without leading zeros
## In linux, strftime("%-d") will give it without leading zeros
def generate_report_url(datePublished):
    urlbase = "https://www.covid19lst.org/post/"
    urlend = "daily-covid-19-lst-report"
    is_windows = sys.platform.startswith('win')
    if is_windows==True:
        reporturl = urlbase+datePublished.strftime("%B").lower()+"-"+datePublished.strftime("%#d")+"-"+urlend
    else:
        reporturl = urlbase+datePublished.strftime("%B").lower()+"-"+datePublished.strftime("%-d")+"-"+urlend
    return(reporturl)

In [9]:
def generate_report_meta(filelist):
    report_pmid_df = pandas.DataFrame(columns=['_id','name','identifier','url'])
    curatedByObject = generate_curator()
    author = generate_author()
    for eachfile in filelist:
        reportdate = eachfile[0:4]+'.'+eachfile[4:6]+'.'+eachfile[6:8]
        datePublished = datetime.fromisoformat(eachfile[0:4]+'-'+eachfile[4:6]+'-'+eachfile[6:8])
        name = "Covid-19 LST Report "+reportdate
        reporturl = generate_report_url(datePublished)
        report_id = 'lst'+reportdate
        pmidlist,doilist = parse_pdf(eachfile)
        try:
            basedOndf,missing = merge_meta(pmidlist,doilist)
            reportlinkdf = basedOndf[['_id','url']]
            reportlinkdf['identifier']=report_id
            reportlinkdf['url']=reporturl
            reportlinkdf['name']=name
            report_pmid_df = pandas.concat(([report_pmid_df,reportlinkdf]),ignore_index=True)
            report_pmid_df.drop_duplicates(keep='first',inplace=True)
            report_pmid_df.to_csv('data/report_pmid_df.txt',sep='\t',header=True)
            save_missing(missing)
            abstract = generate_abstract(basedOndf['_id'].unique().tolist())
            metadict = {"@context": {"schema": "http://schema.org/", "outbreak": "https://discovery.biothings.io/view/outbreak/"}, 
                        "@type": "Publication", "journalName": "COVID-19 LST Daily Summary Reports", "journalNameAbbreviation": "covid19LST", 
                        "publicationType": "Review", "license":"(CC BY-NC-SA 4.0) (http://creativecommons.org/licenses/by-nc-sa/4.0/)",
                        "_id":report_id,"curatedBy": curatedByObject,"abstract": abstract, "name": name, 
                        "datePublished": datePublished.strftime("%Y-%m-%d"),"url": reporturl,"author":[author], 
                        "isBasedOn":basedOndf.to_dict('records')}
            yield(metadict)
        except:
            save_missing(list(report_id))

In [10]:
## Unit test

dumpdir = 'data/reports/'
filelist = os.listdir(dumpdir)
testlist = filelist[0:2]
metadict = generate_report_meta(testlist)
for eachdict in metadict:
    print(eachdict)

{'@context': {'schema': 'http://schema.org/', 'outbreak': 'https://discovery.biothings.io/view/outbreak/'}, '@type': 'Publication', 'journalName': 'COVID-19 LST Daily Summary Reports', 'journalNameAbbreviation': 'covid19LST', 'publicationType': 'Review', 'license': '(CC BY-NC-SA 4.0) (http://creativecommons.org/licenses/by-nc-sa/4.0/)', '_id': 'lst2020.04.01', 'curatedBy': {'@type': 'Organization', 'identifier': 'covid19LST', 'url': 'https://www.covid19lst.org/', 'name': 'COVID-19 Literature Surveillance Team', 'affiliation': [], 'curationDate': '2021-02-25'}, 'abstract': 'Analytical reviews on the level of evidence presented in publications. This report specifically covers the following publications: pmid32208987, pmid32107119, pmid32223077, pmid32232980, pmid32232648, pmid32232429, pmid32233018, pmid32215952, pmid32232986, pmid32232976, pmid32232837', 'name': 'Covid-19 LST Report 2020.04.01', 'datePublished': '20-04-01', 'url': 'https://www.covid19lst.org/post/april-1-daily-covid-19-

{'@context': {'schema': 'http://schema.org/', 'outbreak': 'https://discovery.biothings.io/view/outbreak/'}, '@type': 'Publication', 'journalName': 'COVID-19 LST Daily Summary Reports', 'journalNameAbbreviation': 'covid19LST', 'publicationType': 'Review', 'license': '(CC BY-NC-SA 4.0) (http://creativecommons.org/licenses/by-nc-sa/4.0/)', '_id': 'lst2020.04.02', 'curatedBy': {'@type': 'Organization', 'identifier': 'covid19LST', 'url': 'https://www.covid19lst.org/', 'name': 'COVID-19 Literature Surveillance Team', 'affiliation': [], 'curationDate': '2021-02-25'}, 'abstract': 'Analytical reviews on the level of evidence presented in publications. This report specifically covers the following publications: pmid32229155, pmid32229706, pmid32233040, pmid32229732, pmid32231349, pmid32229257, pmid32226289, pmid32237278', 'name': 'Covid-19 LST Report 2020.04.02', 'datePublished': '20-04-02', 'url': 'https://www.covid19lst.org/post/april-2-daily-covid-19-lst-report', 'author': [{'@type': 'Organiz

## Download PDFs from google drive

https://drive.google.com/drive/folders/1603ahBNdt1SnSaYYBE-G8SA6qgRTQ6fF

but only if they were uploaded after a specific date. The reason for this is that prior to 2020.09.11, there was variability in the file naming and file names were sometimes incorrect (202080 instead of 202008). These have been manually corrected in the initial dump, but are otherwise still incorrect in the google drive.

Note that this script uses the googledrive api which requires authentification even when accessing a public google drive. To fulfill this requirement without needing to manually log in, credentials from a service account are needed  The googledrive API is only used to read the files in the drive so that the newest ones (by date) can be identified, and their id's taken.

Additionally the the pydrive2 library (use to access the google drive api) sometimes has trouble finding the client_secrets.json file, so you may need to manually point to it.

The downloader uses the GoogleDriveDownloader library which is based off of requests and should not require the google drive api.

In [12]:
## This function identifies files uploaded after 2020.09.11 that have NOT yet been downloaded
## Note that this is the function if a service account is not available. It requires a login

def check_google():
    from pydrive2.auth import GoogleAuth
    from pydrive2.drive import GoogleDrive

    GoogleAuth.DEFAULT_SETTINGS['client_config_file'] = 'client_secrets.json' ##point to secrets file location
    gauth = GoogleAuth()
    #gauth.LocalWebserverAuth()

    drive = GoogleDrive(gauth)
    file_id = '1603ahBNdt1SnSaYYBE-G8SA6qgRTQ6fF'
    file_list = drive.ListFile({'q': "'%s' in parents and trashed=false" % file_id}).GetList()
    
    df = pandas.DataFrame(file_list)
    dfclean = df[['createdDate','id','title']].copy()
    dfclean['date'] = pandas.to_datetime(dfclean['createdDate'],format='%Y-%m-%d', errors='coerce')
    lastupdate = dfclean.loc[dfclean['createdDate']=='2020-09-11T01:53:29.639Z'].iloc[0]['date']
    dfnew = dfclean.loc[dfclean['date']>lastupdate]
    
    all_files = os.listdir('data/reports/')
    new_files = [item for item in dfnew['title'].unique().tolist() if item not in all_files]
    reportdf = dfnew.loc[dfnew['title'].isin(new_files)]
    return(reportdf)

In [11]:
## This function identifies files uploaded after 2020.09.11 that have NOT yet been downloaded
## Note that this is the function if a service account IS available. 
def check_google():
    from pydrive2.auth import GoogleAuth
    from pydrive2.drive import GoogleDrive
    from pydrive2.auth import ServiceAccountCredentials
    
    gauth = GoogleAuth()
    scope = ['https://www.googleapis.com/auth/drive']
    gauth.credentials = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)
    drive = GoogleDrive(gauth)
    file_id = '1603ahBNdt1SnSaYYBE-G8SA6qgRTQ6fF'
    file_list = drive.ListFile({'q': "'%s' in parents and trashed=false" % file_id}).GetList()
    
    df = pandas.DataFrame(file_list)
    dfclean = df[['createdDate','id','title']].copy()
    dfclean['date'] = pandas.to_datetime(dfclean['createdDate'],format='%Y-%m-%d', errors='coerce')
    lastupdate = dfclean.loc[dfclean['createdDate']=='2020-09-11T01:53:29.639Z'].iloc[0]['date']
    dfnew = dfclean.loc[dfclean['date']>lastupdate]
    
    all_files = os.listdir('data/reports/')
    new_files = [item for item in dfnew['title'].unique().tolist() if item not in all_files]
    reportdf = dfnew.loc[dfnew['title'].isin(new_files)]
    return(reportdf)

In [12]:
#### This is the function to actually conduct the download
## Note that the report date in the title is used for generating the corresponding covid19 LST report url
## For this reason, special reports cannot be automated and will be exempt from downloading
def download_reports(reportdf):
    from google_drive_downloader import GoogleDriveDownloader as gdd
    notdownloaded = 0
    for i in range(len(reportdf)):
        title = reportdf.iloc[i]['title']
        eachid = reportdf.iloc[i]['id']
        try:
            date_title = int(title[0:6])
            gdd.download_file_from_google_drive(file_id=eachid,
                                                dest_path='data/reports/'+title,
                                                unzip=False)
        except:
            notdownloaded = notdownloaded+1   

In [13]:
def load_annotations():
    reportdf = check_google()
    download_reports(reportdf)
    dumpdir = 'data/reports/'
    filelist = os.listdir(dumpdir)
    metadict = generate_report_meta(filelist)
    yield from(metadict)

In [14]:
##### Unit test

reportdf = check_google()
download_reports(reportdf)
dumpdir = 'data/reports/'
filelist = os.listdir(dumpdir)
metadict = generate_report_meta(filelist)
for eachmeta in metadict:
    print(eachmeta)

{'@context': {'schema': 'http://schema.org/', 'outbreak': 'https://discovery.biothings.io/view/outbreak/'}, '@type': 'Publication', 'journalName': 'COVID-19 LST Daily Summary Reports', 'journalNameAbbreviation': 'covid19LST', 'publicationType': 'Review', 'license': '(CC BY-NC-SA 4.0) (http://creativecommons.org/licenses/by-nc-sa/4.0/)', '_id': 'lst2020.04.01', 'curatedBy': {'@type': 'Organization', 'identifier': 'covid19LST', 'url': 'https://www.covid19lst.org/', 'name': 'COVID-19 Literature Surveillance Team', 'affiliation': [], 'curationDate': '2021-02-25'}, 'abstract': 'Analytical reviews on the level of evidence presented in publications. This report specifically covers the following publications: pmid32208987, pmid32107119, pmid32223077, pmid32232980, pmid32232648, pmid32232429, pmid32233018, pmid32215952, pmid32232986, pmid32232976, pmid32232837', 'name': 'Covid-19 LST Report 2020.04.01', 'datePublished': '20-04-01', 'url': 'https://www.covid19lst.org/post/april-1-daily-covid-19-

{'@context': {'schema': 'http://schema.org/', 'outbreak': 'https://discovery.biothings.io/view/outbreak/'}, '@type': 'Publication', 'journalName': 'COVID-19 LST Daily Summary Reports', 'journalNameAbbreviation': 'covid19LST', 'publicationType': 'Review', 'license': '(CC BY-NC-SA 4.0) (http://creativecommons.org/licenses/by-nc-sa/4.0/)', '_id': 'lst2020.04.02', 'curatedBy': {'@type': 'Organization', 'identifier': 'covid19LST', 'url': 'https://www.covid19lst.org/', 'name': 'COVID-19 Literature Surveillance Team', 'affiliation': [], 'curationDate': '2021-02-25'}, 'abstract': 'Analytical reviews on the level of evidence presented in publications. This report specifically covers the following publications: pmid32229155, pmid32229706, pmid32233040, pmid32229732, pmid32231349, pmid32229257, pmid32226289, pmid32237278', 'name': 'Covid-19 LST Report 2020.04.02', 'datePublished': '20-04-02', 'url': 'https://www.covid19lst.org/post/april-2-daily-covid-19-lst-report', 'author': [{'@type': 'Organiz

{'@context': {'schema': 'http://schema.org/', 'outbreak': 'https://discovery.biothings.io/view/outbreak/'}, '@type': 'Publication', 'journalName': 'COVID-19 LST Daily Summary Reports', 'journalNameAbbreviation': 'covid19LST', 'publicationType': 'Review', 'license': '(CC BY-NC-SA 4.0) (http://creativecommons.org/licenses/by-nc-sa/4.0/)', '_id': 'lst2020.04.03', 'curatedBy': {'@type': 'Organization', 'identifier': 'covid19LST', 'url': 'https://www.covid19lst.org/', 'name': 'COVID-19 Literature Surveillance Team', 'affiliation': [], 'curationDate': '2021-02-25'}, 'abstract': 'Analytical reviews on the level of evidence presented in publications. This report specifically covers the following publications: pmid32229322, pmid32234804, pmid32238223, pmid32101510, pmid32240128, pmid32233876, pmid32241022, pmid32238232', 'name': 'Covid-19 LST Report 2020.04.03', 'datePublished': '20-04-03', 'url': 'https://www.covid19lst.org/post/april-3-daily-covid-19-lst-report', 'author': [{'@type': 'Organiz

{'@context': {'schema': 'http://schema.org/', 'outbreak': 'https://discovery.biothings.io/view/outbreak/'}, '@type': 'Publication', 'journalName': 'COVID-19 LST Daily Summary Reports', 'journalNameAbbreviation': 'covid19LST', 'publicationType': 'Review', 'license': '(CC BY-NC-SA 4.0) (http://creativecommons.org/licenses/by-nc-sa/4.0/)', '_id': 'lst2020.04.04', 'curatedBy': {'@type': 'Organization', 'identifier': 'covid19LST', 'url': 'https://www.covid19lst.org/', 'name': 'COVID-19 Literature Surveillance Team', 'affiliation': [], 'curationDate': '2021-02-25'}, 'abstract': 'Analytical reviews on the level of evidence presented in publications. This report specifically covers the following publications: pmid32245770, pmid32242947, pmid32247016, pmid32246503, pmid32243818, pmid32245830, pmid32246782, pmid32246117, pmid32243777, pmid32243813, pmid32246119, pmid32246783, pmid32247017, pmid32245769, pmid32245847, pmid32246915, pmid32243608, pmid32241761, pmid32243607, pmid32243599, pmid32243

{'@context': {'schema': 'http://schema.org/', 'outbreak': 'https://discovery.biothings.io/view/outbreak/'}, '@type': 'Publication', 'journalName': 'COVID-19 LST Daily Summary Reports', 'journalNameAbbreviation': 'covid19LST', 'publicationType': 'Review', 'license': '(CC BY-NC-SA 4.0) (http://creativecommons.org/licenses/by-nc-sa/4.0/)', '_id': 'lst2020.04.06', 'curatedBy': {'@type': 'Organization', 'identifier': 'covid19LST', 'url': 'https://www.covid19lst.org/', 'name': 'COVID-19 Literature Surveillance Team', 'affiliation': [], 'curationDate': '2021-02-25'}, 'abstract': 'Analytical reviews on the level of evidence presented in publications. This report specifically covers the following publications: pmid32247925, pmid32247322, pmid32247323, pmid32247381, pmid32247926, pmid32247931, pmid32247927, pmid32247928, pmid32247326, pmid32248565, pmid32247324, pmid32247320', 'name': 'Covid-19 LST Report 2020.04.06', 'datePublished': '20-04-06', 'url': 'https://www.covid19lst.org/post/april-6-d

...

{'@context': {'schema': 'http://schema.org/', 'outbreak': 'https://discovery.biothings.io/view/outbreak/'}, '@type': 'Publication', 'journalName': 'COVID-19 LST Daily Summary Reports', 'journalNameAbbreviation': 'covid19LST', 'publicationType': 'Review', 'license': '(CC BY-NC-SA 4.0) (http://creativecommons.org/licenses/by-nc-sa/4.0/)', '_id': 'lst2020.09.04', 'curatedBy': {'@type': 'Organization', 'identifier': 'covid19LST', 'url': 'https://www.covid19lst.org/', 'name': 'COVID-19 Literature Surveillance Team', 'affiliation': [], 'curationDate': '2021-02-25'}, 'abstract': 'Analytical reviews on the level of evidence presented in publications. This report specifically covers the following publications: pmid32876905, pmid32840571, pmid32859963, pmid32870481, pmid32841429, pmid32861333, pmid32871617, pmid32866168, pmid32877576', 'name': 'Covid-19 LST Report 2020.09.04', 'datePublished': '20-09-04', 'url': 'https://www.covid19lst.org/post/september-4-daily-covid-19-lst-report', 'author': [

{'@context': {'schema': 'http://schema.org/', 'outbreak': 'https://discovery.biothings.io/view/outbreak/'}, '@type': 'Publication', 'journalName': 'COVID-19 LST Daily Summary Reports', 'journalNameAbbreviation': 'covid19LST', 'publicationType': 'Review', 'license': '(CC BY-NC-SA 4.0) (http://creativecommons.org/licenses/by-nc-sa/4.0/)', '_id': 'lst2020.09.08', 'curatedBy': {'@type': 'Organization', 'identifier': 'covid19LST', 'url': 'https://www.covid19lst.org/', 'name': 'COVID-19 Literature Surveillance Team', 'affiliation': [], 'curationDate': '2021-02-25'}, 'abstract': 'Analytical reviews on the level of evidence presented in publications. This report specifically covers the following publications: pmid32852928, pmid32399563, pmid32838945, pmid32813543, pmid32845733, pmid32810218, pmid32865241, pmid32835427, pmid32841356, pmid32804187, pmid32810303, pmid32845869, pmid32703420, pmid32652713, pmid32790653, pmid32871003, pmid32876900', 'name': 'Covid-19 LST Report 2020.09.08', 'datePub

{'@context': {'schema': 'http://schema.org/', 'outbreak': 'https://discovery.biothings.io/view/outbreak/'}, '@type': 'Publication', 'journalName': 'COVID-19 LST Daily Summary Reports', 'journalNameAbbreviation': 'covid19LST', 'publicationType': 'Review', 'license': '(CC BY-NC-SA 4.0) (http://creativecommons.org/licenses/by-nc-sa/4.0/)', '_id': 'lst2020.09.10', 'curatedBy': {'@type': 'Organization', 'identifier': 'covid19LST', 'url': 'https://www.covid19lst.org/', 'name': 'COVID-19 Literature Surveillance Team', 'affiliation': [], 'curationDate': '2021-02-25'}, 'abstract': 'Analytical reviews on the level of evidence presented in publications. This report specifically covers the following publications: pmid32894756, pmid32840240, pmid32530583, pmid32860437, pmid32786309, pmid32790309, pmid32863359, pmid32870707, pmid32870462', 'name': 'Covid-19 LST Report 2020.09.10', 'datePublished': '20-09-10', 'url': 'https://www.covid19lst.org/post/september-10-daily-covid-19-lst-report', 'author': 

{'@context': {'schema': 'http://schema.org/', 'outbreak': 'https://discovery.biothings.io/view/outbreak/'}, '@type': 'Publication', 'journalName': 'COVID-19 LST Daily Summary Reports', 'journalNameAbbreviation': 'covid19LST', 'publicationType': 'Review', 'license': '(CC BY-NC-SA 4.0) (http://creativecommons.org/licenses/by-nc-sa/4.0/)', '_id': 'lst2020.09.11', 'curatedBy': {'@type': 'Organization', 'identifier': 'covid19LST', 'url': 'https://www.covid19lst.org/', 'name': 'COVID-19 Literature Surveillance Team', 'affiliation': [], 'curationDate': '2021-02-25'}, 'abstract': 'Analytical reviews on the level of evidence presented in publications. This report specifically covers the following publications: pmid32897389, pmid32897539, pmid32894287, pmid32873357, pmid32897660, pmid32830286, pmid32895744, pmid32894747, pmid32878584, pmid32881018, pmid32894753, pmid32897543', 'name': 'Covid-19 LST Report 2020.09.11', 'datePublished': '20-09-11', 'url': 'https://www.covid19lst.org/post/september

PdfReadError: Could not read malformed PDF file

In [51]:
print(filelist[82])

20200812 The COVID-19 Daily Report.pdf


In [103]:
##### Unit test
##https://www.covid19lst.org/post/september-11-daily-covid-19-lst-report
#dt.strftime("%A, %d. %B %Y %I:%M%p")
#'Tuesday, 21. November 2006 04:30PM'

report_pmid_df = pandas.DataFrame(columns=['_id','name','identifier','url'])
curatedByObject = generate_curator()
author = curatedByObject.copy()
author.pop("curationDate")
#curatedByObject.pop('members')

badpdfs = []

for eachfile in filelist[83:]:
    reportdate = eachfile[0:4]+'.'+eachfile[4:6]+'.'+eachfile[6:8]
    datePublished = datetime.fromisoformat(eachfile[0:4]+'-'+eachfile[4:6]+'-'+eachfile[6:8])
    name = "Covid-19 LST Report "+reportdate
    reporturl = generate_report_url(datePublished)
    report_id = 'lst'+reportdate
    pmidlist,doilist = parse_pdf(eachfile)
    if len(pmidlist)+len(doilist)==0:
        badpdfs.append(eachfile)
    else:
        basedOndf,missing = merge_meta(pmidlist,doilist)
        basedOndf['@type']='Publication'
        reportlinkdf = basedOndf[['_id','url']].copy()
        reportlinkdf['identifier']=report_id
        reportlinkdf['url']=reporturl
        reportlinkdf['name']=name
        report_pmid_df = pandas.concat(([report_pmid_df,reportlinkdf]),ignore_index=True)
        report_pmid_df.drop_duplicates(keep='first',inplace=True)
        report_pmid_df.to_csv('data/report_pmid_df.txt',sep='\t',header=True)
        save_missing(missing)
        abstract = generate_abstract(basedOndf['_id'].unique().tolist())
        metadict = {"@context": {"schema": "http://schema.org/", "outbreak": "https://discovery.biothings.io/view/outbreak/"}, 
                    "@type": "Publication", "journalName": "COVID-19 LST Daily Summary Reports", "journalNameAbbreviation": "covid19LST", 
                    "publicationType": "Review", "license":"(CC BY-NC-SA 4.0) (http://creativecommons.org/licenses/by-nc-sa/4.0/)",
                    "_id":report_id,"curatedBy": curatedByObject,"abstract": abstract, "name": name, 
                    "datePublished": datePublished.strftime("%Y-%m-%d"),"url": reporturl, 
                    "isBasedOn":basedOndf.to_dict('records')}
        with open("results/json/"+name+".json", "w") as outfile:  
            json.dump(metadict, outfile) 

print(len(badpdfs))    

66


In [101]:

### Batch fetch pmid meta
def convert_dois(doilist):
    doistring = '"' + '","'.join(doilist) + '"'
    r = requests.post("https://api.outbreak.info/resources/query/", params = {'q': doistring, 'scopes': 'doi', 'fields': '_id,name,url,doi'})
    if r.status_code == 200:
        rawresult = pandas.read_json(r.text)
        if 'notfound' in rawresult.columns:
            check = rawresult.loc[(rawresult['notfound']==1.0)|(rawresult['notfound']==True)]
            if len(check)==len(doilist):
                cleanresult = pandas.DataFrame(columns=['_id','name','url'])
                missing = doilist            
            else:
                no_dups = rawresult[rawresult['query']==rawresult['doi']]
                cleanresult = no_dups[['_id','name','url']].loc[~no_dups['_id'].isin(check['_id'].tolist())].copy()
                missing = [x for x in doilist if x not in cleanresult['doi'].unique().tolist()]
        else:
            no_dups = rawresult[rawresult['query']==rawresult['doi']]
            cleanresult = no_dups[['_id','name','url']]
            missing = [x for x in doilist if x not in cleanresult['doi'].unique().tolist()]
    return(cleanresult, missing)

### Batch fetch pmid meta
def get_pmid_meta(pmidlist):
    pmidstring = '"' + '","'.join(pmidlist) + '"'
    r = requests.post("https://api.outbreak.info/resources/query/", params = {'q': pmidstring, 'scopes': '_id', 'fields': '_id,name,url'})
    if r.status_code == 200:
        rawresult = pandas.read_json(r.text)
        no_dups = rawresult[rawresult['query']==rawresult['_id']]
        if 'notfound' in rawresult.columns:
            check = rawresult.loc[(rawresult['notfound']==1.0)|(rawresult['notfound']==True)]
            if len(check)==len(pmidlist):
                cleanresult = pandas.DataFrame(columns=['_id','name','url'])
                missing = pmidlist            
            else:
                cleanresult = no_dups[['_id','name','url']].loc[~no_dups['_id'].isin(check['_id'].tolist())].copy()
        else:
            cleanresult = no_dups[['_id','name','url']]
        missing = [x for x in pmidlist if x not in cleanresult['_id'].unique().tolist()]
    return(cleanresult, missing)

In [71]:
print(len(pmidlist),len(doilist))

15 1


In [100]:
doistring = '"' + '","'.join(doilist) + '"'
r = requests.post("https://api.outbreak.info/resources/query/", params = {'q': doistring, 'scopes': 'doi', 'fields': '_id,name,url,doi'})
if r.status_code == 200:
    rawresult = pandas.read_json(r.text)
    if 'notfound' in rawresult.columns:
        check = rawresult.loc[(rawresult['notfound']==1.0)|(rawresult['notfound']==True)]
        if len(check)==len(doilist):
            cleanresult = pandas.DataFrame(columns=['_id','name','url'])
            missing = doilist            
        else:
            no_dups = rawresult[rawresult['query']==rawresult['doi']]
            cleanresult = no_dups[['_id','name','url']].loc[~no_dups['_id'].isin(check['_id'].tolist())].copy()
            missing = [x for x in doilist if x not in cleanresult['doi'].unique().tolist()]
    else:
        no_dups = rawresult[rawresult['query']==rawresult['doi']]
        cleanresult = no_dups[['_id','name','url']]
        missing = [x for x in doilist if x not in cleanresult['doi'].unique().tolist()]


In [99]:
pmidstring = '"' + '","'.join(pmidlist) + '"'
r = requests.post("https://api.outbreak.info/resources/query/", params = {'q': pmidstring, 'scopes': '_id', 'fields': '_id,name,url'})
if r.status_code == 200:
    rawresult = pandas.read_json(r.text)
    no_dups = rawresult[rawresult['query']==rawresult['_id']]
    if 'notfound' in rawresult.columns:
        check = rawresult.loc[(rawresult['notfound']==1.0)|(rawresult['notfound']==True)]
        if len(check)==len(pmidlist):
            cleanresult = pandas.DataFrame(columns=['_id','name','url'])
            missing = pmidlist            
        else:
            cleanresult = no_dups[['_id','name','url']].loc[~no_dups['_id'].isin(check['_id'].tolist())].copy()
            missing = [x for x in pmidlist if x not in cleanresult['_id'].unique().tolist()]
    else:
        cleanresult = no_dups[['_id','name','url']]
        missing = [x for x in pmidlist if x not in cleanresult['_id'].unique().tolist()]


print(cleanresult)

             _id                                               name  \
0   pmid32749458  The International Health Regulations (2005) an...   
1   pmid32762140  Circulating endothelial progenitors are increa...   
2   pmid32749759  Exploring the growth of COVID-19 cases using e...   
3   pmid32761898  A systematic review and meta-analysis of child...   
4   pmid32761839  Understanding the traumatic experiences of hea...   
5   pmid32761993  Longitudinal changes of liver function and hep...   
6   pmid32761492  Successful management of seven cases of critic...   
7   pmid32770758  Gender differences in the battle against COVID...   
8   pmid32752979  Naturally occurring SARS-CoV-2 gene deletions ...   
9   pmid32744591  The Uncertain Role of Corticosteroids in the T...   
10  pmid32628534  Clinical Validity of Serum Antibodies to SARS-...   
11  pmid32762116  Technology as a coping tool during the coronav...   
12  pmid32687485  A 46-Year-Old Woman Who Presented with Diabeti...   
13  pm

In [72]:
if len(doilist)>0:
    doianns,missing_dois = convert_dois(doilist)
    doicheck = True
else:
    doicheck = False
    missing_dois = None
if len(pmidlist)>0:
    pmidanns,missing_pmids = get_pmid_meta(pmidlist)
    pmidcheck = True
else:
    pmidcheck = False
    missing_pmids = None
if doicheck==True and pmidcheck==True:
    basedOndf = pandas.concat((pmidanns,doianns),ignore_index=True)
elif doicheck==True and pmidcheck==False:
    basedOndf = doianns
elif doicheck==False and pmidcheck==True:
    basedOndf = pmidanns     
if missing_pmids!=None and missing_dois!=None:
    missing = list(set(missing_pmids).union(set(missing_dois)))
elif missing_pmids==None and missing_dois!=None:
    missing = missing_dois
elif missing_pmids!=None and missing_dois==None:
    missing = missing_pmids
else:
    missing = None

print(pmidcheck)

KeyError: 'notfound'